# Notebook 8 - Représentation des connaissances

CSI4506 Intelligence Artificielle  
Automne 2021 \
Versions 1 (2020) préparée par Julian Templeton, Caroline Barrière et Joel Muteba.  Version 2 (2021) modifiée par Caroline Barrière.

***INTRODUCTION***:  

Lors de la lecture de texte, comprendre les type d'entités utilisées dans le texte permet d'inférer des informations supplémentaires sur ces entités.  Par exemple, si un texte mentionne *Canada*, le fait de savoir que c'est une GPE (entité géo-politique), nous indique déjà que cette entité a une supercifie, une population, etc.  Grâce à l'utilisation de la reconnaissance d'entités nommées (Named Entity Recognition, NER), nous sommes en mesure de déterminer si une entité est une personne, une organisation, un pays, ... 

Lors de l'exploration de texte en version électronique, nous voyons aussi occasionnellement que les entités ont des liens cliquables vers des pages Web avec plus d'informations sur l'entité. Il s'agit d'une forme d'amélioration du texte pour permettre aux lecteurs d'accéder facilement à des informations supplémentaires.  Si nous prenons encore l'exemple de *Canada*, si nous le transformons en [Canada](https://en.wikipedia.org/wiki/Canada), à l'aide du linking d'entités (entity linking) nous accédons à d'avantage d'informations. 

Dans ce notebook, nous revisiterons l'ensemble de données d'actualités liées à Covid-19 du notebook 7 pour explorer comment nous pouvons améliorer les résultats de NER de spaCy et aider à la compréhension des articles de presse grâce à l'utilisation du linking d'entités. Cela se fera en trois parties, soit 

(1) nous explorerons d'abord les résultats du NER de spaCy \
(2) nous utiliserons la cohérence du texte pour un post-traitement au NER de spaCy\
(3) puis nous effectuons des ajouts (*enrichissement*) au texte avec le linking d'entités.

Ce notebook utilise des bibliothèques qui ont été utilisées dans les notebooks précédents, notamment spaCy et pandas. 

***DEVOIR***:  

Parcourez le notebook en exécutant chaque cellule, une à la fois. \
Recherchez **(TO DO)** pour les tâches que vous devez effectuer. Ne modifiez pas le code en dehors des questions auxquelles vous êtes invité à répondre à moins que cela ne vous soit spécifiquement demandé. Une fois que vous avez terminé, signez le notebook (à la fin du notebook), renommez-le *NumEtudiant-NomFamille-Notebook8.ipynb* et soumettez-le.

*Le notebook sera noté le 30.  \
Chaque **(TO DO)** est associé à un certain nombre de points.*
***

In [1]:
# Before starting we will import every module that we will be using
import spacy
import pandas as pd

In [2]:
# The core spacy object can be used for tokenization, lemmatization, POS Tagging, NER ...
# Note that this is specifically for the English language and requires the English package to be installed
# via pip to work as intended.

# sp = spacy.load('en')

# If the above causes an error then install the package as below
# !spacy download en_core_web_sm
sp = spacy.load('en_core_web_sm')

Comme pour le dernier notebook, l'ensemble de données est fourni sur Brightspace (Module 8) avec ce notebook, mais les détails le concernant peuvent être trouvés [ici](https://www.kaggle.com/ryanxjhan/cbc-news-coronavirus-articles-march-26?select=news.csv). La première chose que nous allons faire, comme d'habitude, est de charger le fichier dans un dataframe pandas.

In [3]:
# Read the dataset, show top ten rows
df = pd.read_csv("news.csv")
df.head(10)

,Unnamed: 0,authors,title,publish_date,description,text,url
0,0,[],'More vital now:' Gay-straight alliances go vi...,2020-05-03 1:30,Lily Overacker and Laurell Pallot start each g...,Lily Overacker and Laurell Pallot start each g...,https://www.cbc.ca/news/canada/calgary/gay-str...
1,1,[],Scientists aim to 'see' invisible transmission...,2020-05-02 8:00,Some researchers aim to learn more about how t...,"This is an excerpt from Second Opinion, a week...",https://www.cbc.ca/news/technology/droplet-tra...
2,2,['The Canadian Press'],Coronavirus: What's happening in Canada and ar...,2020-05-02 11:28,Canada's chief public health officer struck an...,The latest: The lives behind the numbers: Wha...,https://www.cbc.ca/news/canada/coronavirus-cov...
3,3,[],"B.C. announces 26 new coronavirus cases, new c...",2020-05-02 18:45,B.C. provincial health officer Dr. Bonnie Henr...,B.C. provincial health officer Dr. Bonnie Henr...,https://www.cbc.ca/news/canada/british-columbi...
4,4,[],"B.C. announces 26 new coronavirus cases, new c...",2020-05-02 18:45,B.C. provincial health officer Dr. Bonnie Henr...,B.C. provincial health officer Dr. Bonnie Henr...,https://www.cbc.ca/news/canada/british-columbi...
5,5,"['Senior Writer', 'Chris Arsenault Is A Senior...",Brazil has the most confirmed COVID-19 cases i...,2020-05-02 8:00,"From describing coronavirus as a ""little flu,""...","With infection rates spiralling, some big city...",https://www.cbc.ca/news/world/brazil-has-the-m...
6,6,['Cbc News'],The latest on the coronavirus outbreak for May 1,2020-05-01 20:43,The latest on the coronavirus outbreak from CB...,Coronavirus Brief (CBC) Canada is officiall...,https://www.cbc.ca/news/the-latest-on-the-coro...
7,7,['Cbc News'],Coronavirus: What's happening in Canada and ar...,2020-05-01 11:51,Nova Scotia announced Friday it is immediately...,The latest: The lives behind the numbers: Wha...,https://www.cbc.ca/news/canada/coronavirus-cov...
8,8,"['Senior Writer', ""Adam Miller Is Senior Digit...",Did the WHO mishandle the global coronavirus p...,2020-04-30 8:00,The World Health Organization has come under f...,The World Health Organization has come under f...,https://www.cbc.ca/news/health/coronavirus-who...
9,9,['Thomson Reuters'],Armed people in Michigan's legislature protest...,2020-04-30 21:37,"Hundreds of protesters, some armed, gathered a...","Hundreds of protesters, some armed, gathered a...",https://www.cbc.ca/news/world/protesters-michi...


**PARTIE 1 - Le NER de spaCy**  
  
Commençons par regarder le NER qui est effectué par spaCy. La documentation de SpaCy ne nous dit pas exactement comment leur NER est fait (certainement leur secret commercial), mais nous pouvons au moins regarder les résultats.

Comme nous en avons discuté dans les notebooks précédents, lors de l'évaluation d'un processus, d'un modèle ou d'un outil, nous pouvons faire une évaluation quantitative ou une **évaluation qualitative** des résultats. Dans ce notebook, nous travaillons à un niveau qualitatif, ce qui signifie que nous ne mesurons pas des métriques telles que la précision/rappel, mais imprimons plutôt les résultats de quelques exemples et essayons de comprendre ces résultats.


Vous trouverez ci-dessous la même phrase exemple que dans le dernier Notebook, pour laquelle nous avions examiné l'étiquetage des parties du discours (POS tagging) et d'autres processus linguistiques.  Nous utilisons cette phrase exemple pour illustrer maintenant comment obtenir les prédictions de type NER de spaCy pour les tokens dans un texte.

In [5]:
# Same example from notebook 7, recall that we loop through the iterator found in the .ents property of a parsed sentence
sentence_example = "Government guidelines in Canada recommend that people stay at least two metres away from others as part of physical distancing measures to curb the spread of COVID-19."
sentence_example_content = sp(sentence_example)
# Loop through all tokens that contain a NER type and print the token along with the corresponding NER type
for token in sentence_example_content.ents:
    print("\"" + token.text + "\" is a " + token.label_ )

"Canada" is a GPE
"at least two metres" is a CARDINAL
"COVID-19" is a ORG


**(TO DO) Q1 - 5 points**  

Dans le texte du ***second document*** (index 1) de notre corpus de documents, quels mots sont *PER* (spaCy utilise le type *PERSON*, plutôt que *PER*), *ORG* (Organisation) et *GPE* (entité géopolitique). Vous devez effectuer les opérations suivantes pour cette question:

a) (2 points) Imprimez chaque *PERSON*, *ORG* et *GPE* avec son type NER tel que trouvé par spaCy.

b) (1 points) Est-ce que la majorité des prédictions de spaCy sont correctes? Donnez deux exemples de sorties obtenues en (a) qui sont incorrectes selon vous.

c) (2 points) Il arrive parfois que des problèmes avec les prédictions de type NER proviennent d'erreurs dans des étapes antérieures dans la pipeline TAL (e.g. tokenization, POS tagging).  Utilisez 2 exemples de sorties obtenues en (a) pour illustrer cette possibilité, et tentez d'offrir un diagnostic (que s'est-il passé?).

In [12]:
# RÉPONSE Q1(a) - 2 points
# Select the second document (index 1)
doc = df["text"][1]
# SHOW PERSON, ORG, GPE
content = sp(doc)
for token in content.ents:
    if(token.label_ == "PERSON" or token.label_ == "ORG" or token.label_ == "GPE"):
        print("\"" + token.text + "\": " + token.label_ )

"COVID-19": PERSON
"the World Health Organization": ORG
"WHO": ORG
"the Public Health Agency": ORG
"Canada": GPE
"W.F. Wells": PERSON
"the Harvard School of Public Health": ORG
"Wells": ORG
"Canada": GPE
"Lydia Bourouiba": PERSON
"the Fluid Dynamics of Disease Transmission Laboratory": ORG
"the Massachusetts Institute of Technology": ORG
"Bourouiba": PERSON
"Mark Loeb": PERSON
"McMaster University": ORG
"RNA": ORG
"Wuhan": GPE
"China": GPE
"Nebraska": GPE
"Canada": GPE
"COVID-19": ORG
"Gary Moore/CBC": PERSON
"Allison McGeer": PERSON
"Sinai Health": ORG
"Toronto": GPE
"COVID-19": PERSON
"McGeer": ORG
"McGeer": ORG
"Bourouiba": PERSON
"Bourouiba": PERSON
"Credit Lydia Bourouiba/MIT/JAMA Networks": ORG
"Samira Mubareka": PERSON
"Toronto": GPE
"Bourouiba": PERSON
"COVID-19": ORG
"McMaster": PERSON
"N95": ORG
"U.S.": GPE
"Justin Trudeau": PERSON
"Mubareka": PERSON
"the New England Journal of Medicine": ORG
"the U.S. National Institutes of Health": ORG
"U.S. National Institutes of Health": 

**RÉPONSE Q1(b) - 1 point**   
"Covid-19" est parfois associé à une organisation ou une personne ce qui n'est pas le cas.
"McGeer" est associé à une organisation, mais est en réalité une personne.
La majorité des prédictions de spaCY sont correctes.


**RÉPONSE Q1 (c) - 2 points**   
"Gary Moore/CBC" est une erreur de tokenization, car Gary Moore est une personne et CBC est une organisation. Cette erreur est sans doute dû au "/" entre les deux mots.

"McGeer" est identifié comme une organisation, mais fait fort probablement référence à une personne, donc il y a eu une erreur de POS tagging. Cette erreur doit être dû au fait que le prénom n'était pas présent avant le nom "McGeer", donc spaCY à traité "McGeer" comme le nom d'une organisation. De plus, spaCY ne tient pas compte de l'entièreté du document.

**PARTIE 2 - Cohérence du texte et chaînes de coréférences**  

Comme vous avez vu, les résultats du spaCy NER sont très bons, mais pas parfait.  Un problème principal avec NER (pas seulement dans spaCy mais dans de nombreux outils) est que l'annotation est effectuée une entité à la fois sans tenir compte du document global.

Mais en regardant l'ensemble du document, et sachant que le texte est généralement cohérent, nous pouvons effectuer un post-traitement dans le module NER de spaCy et corriger certaines erreurs. Par texte cohérent, nous entendons, par exemple, que si une personne est désignée avec un nom particulier, par ex. *McGeer*, il y a de fortes chances qu'à chaque fois que l'on voit *McGeer* dans le document, ce soit la même personne.  Toutes les mentions *McGeer* formeraient une chaîne de coréférences vers la même entité.  Il est donc peu probable que *McGeer* soit une fois une personne et une fois une organisation. Ce n'est pas toujours vrai, il existe de nombreux contre-exemples, mais c'est une hypothèse courante. Cette idée est même le sujet d'un article de la PNL plus ancien et très cité intitulé « One sense per discourse » (Gale et al. 1992).

Avec cette idée de "One sense per discourse", nous explorerons deux stratégies différentes pour utiliser la cohérence du texte pour post-traiter la sortie du module spaCy NER.

La première stratégie (*explorée en Q2 / Q3*) est de trouver, parmi tous les types de NER assignés, lequel est le plus fréquent. Par exemple, l'entité *Bourouiba* s'est vu attribuer 1 fois ORG et 2 fois PERSON, donc ces informations peuvent être utilisées pour modifier le type ORG et le changer en PERSON.

La deuxième stratégie (explorée à la Q4) est d'essayer de trouver une forme plus longue dans le texte. Puisque cette forme plus longue devrait être moins ambiguë, nous pouvons l'utiliser pour lever l'ambiguïté des formes plus courtes et plus ambiguës. Par exemple, *Lydia Bourouiba* apparaît dans le texte et se voit attribuer PERSON. Nous pouvons utiliser cette information pour attribuer à d'autres occurrences de la forme abrégée *Bourouiba* le même type PERSON.

Bien sûr, utiliser ces méthodes pour la cohérence du texte ne fonctionnera pas à tous les coups, et introduira malheureusement quelques erreurs... Mais essayons. C'est le but des études empiriques, nous essayons des idées.

Reprenons notre nouvelle utilisée pour Q1, mais cette fois, montrons non seulement GPE, PER, ORG, mais plutôt toutes les entités nommées trouvées par spaCy.

In [14]:
# Select document 2
doc = df["text"][1]
# NER
doc_sp = sp(doc)
# Display all entities from the text along with their index in the .ents iterator and the
# corresponding NER type
for i, token in enumerate(doc_sp.ents):
    print(str(i) + ": \"" + token.text + "\" is a " + token.label_ )

0: "weekly" is a DATE
1: "Saturday" is a DATE
2: "morning" is a TIME
3: "two metres" is a QUANTITY
4: "COVID-19" is a PERSON
5: "the World Health Organization" is a ORG
6: "WHO" is a ORG
7: "more than one metre" is a QUANTITY
8: "the Public Health Agency" is a ORG
9: "Canada" is a GPE
10: "at least two metres" is a QUANTITY
11: "two" is a CARDINAL
12: "2 metres" is a QUANTITY
13: "the 19th century" is a DATE
14: "1934" is a DATE
15: "W.F. Wells" is a PERSON
16: "the Harvard School of Public Health" is a ORG
17: "two metres" is a QUANTITY
18: "Wells" is a ORG
19: "56,000" is a CARDINAL
20: "Canada" is a GPE
21: "Saturday" is a DATE
22: "Lydia Bourouiba" is a PERSON
23: "the Fluid Dynamics of Disease Transmission Laboratory" is a ORG
24: "the Massachusetts Institute of Technology" is a ORG
25: "Bourouiba" is a PERSON
26: "Canadian" is a NORP
27: "Mark Loeb" is a PERSON
28: "McMaster University" is a ORG
29: "RNA" is a ORG
30: "Wuhan" is a GPE
31: "China" is a GPE
32: "Nebraska" is a GPE


**(TO DO) Q2 - 3 points**  
Comme vous pouvez le voir dans les résultats, parfois la même entité s'est vu attribuer différents types d'entités.  Par exemple, *McGeer* est une fois ORG, une fois PERSON, puisque l'algorithme NER regarde phrase par phrase. Dans la fonction suivante, le but sera de trouver tous les types d'entités possibles affectés à une seule entité.

Complétez la définition de la fonction *find_entity_types* ci-dessous. Cette fonction accepte en entrée une entité spaCy spécifique définie par le paramètre *entity* et une liste de toutes les entités spaCy du définies par le paramètre *entities*.

La fonction doit trouver toutes les entités ayant la même forme de surface que *entity* dans l'ensemble *entities*. Pour chaque correspondance entre les entités, ajoutez le type NER trouvé au dictionnaire *type_counts* et mettez à jour la fréquence de ce type.

Le dictionnaire *type_counts* contiendrait par exemple *McGeer* avec ORG = 1, et PERSON = 1, car la fonction a trouvé 2 mentions de *McGeer*, chacune avec un type différent.

In [110]:
# RÉPONSE Q2
def find_entity_types(entity, entities):
    '''
    Given a specific entity and a list of entities, finds all entities from the list that match surface form of the specified
    entity, but that could be of a different type.
    
    Returns the different NER types that have been classified for an entity and the count per NER type
    as a dictionary with the keys as the NER type and the value as the count
    '''
    type_counts = { }
    for token in entities:
        if(token.text == entity.text):
            if(token.label_ in type_counts):
                type_counts[token.label_] += 1  
            else:
                type_counts[token.label_] = 1  
    return type_counts

In [111]:
# Test the above to find the result when checking for the types of the entity 'Bourouiba' 
# from the document loaded above
print("All possible NER types for \"" + doc_sp.ents[59].text + "\" are " + str(find_entity_types(doc_sp.ents[59], doc_sp.ents)))

All possible NER types for "2-metre" are {'QUANTITY': 1}


**(TO DO) Q3 - 2 points**  

Dans la méthode précédente, *find_entity_types*, nous avons trouvé tous les types d'entités possibles pour chaque mention. Par exemple, dans le cas de *McGeer*, c'est une égalité. Mais pour *Bourouiba*, il existe un type ORG et 2 types PERSON, donc le plus courant serait PERSON.

Complétez la définition de la fonction *most_common_type* ci-dessous. Cette fonction accepte en entrée une entité spaCy spécifique définie par le paramètre *entity* et une liste de toutes les entités spaCy définies par le paramètre *entities*.

Remarque: vous pouvez régler les cas d'égalité à votre guise.  Aussi, assurez-vous d'utiliser la méthode *find_entity_types* que vous avez écrite précédemment.


In [155]:
# RÉPONSE Q3 
def most_common_type(entity, entities):
    '''
    Given a specific entity and a list of entities, find the most similar entities and assign the
    NER type to entity based on the most common NER type assigned to entities of the same name (if there
    is a tie, you decide how to handle this).
    
    Returns the most common NER type based on similar entities
    '''
    dic = find_entity_types(entity, entities)
    max_key = max(dic, key=dic.get)
    #entity.label_ = max_key
    return max_key

In [156]:
# Test the above to find the result when checking for the types of the entity 'Bourouiba' 
# from the document loaded above
print("The most common NER type to \"" + doc_sp.ents[59].text + "\" is " + most_common_type(doc_sp.ents[59], doc_sp.ents))

The most common NER type to "2-metre" is QUANTITY



Notre première exploration (au Q2/Q3) portait sur la fréquence d'occurrence. Nous avons supposé que le type d'entité le plus courant pourrait être le bon. Maintenant, nous allons explorer l'idée que la mention la moins ambiguë à une entité pourrait être la bonne. Par exemple, *McGeer* est plus ambigu (forme plus courte) que *Allison McGeer* (forme plus longue). Souvent, la forme la plus longue de référence à une entité est la moins ambiguë. Mais parce que cette forme est longue à écrire, nous l'utilisons souvent avec parcimonie dans un texte (peut-être une seule fois) et les mentions subséquentes de la même entité utiliseront la forme courte. Par exemple, le texte peut mentionner *Allison McGeer* une fois, puis utiliser la forme abrégée *McGeer* pour faire référence à la même personne plusieurs fois dans le document.

Dans les vidéos du cours, nous avons parlé de chaîne de coréférences.  Ainsi, une chaîne contient des mentions longues et courtes, référant toutes à la même entités.

La forme plus longue est souvent appelée forme normalisée, et c'est une forme que nous sommes susceptibles de trouver dans une ressource externe. Nous verrons dans la partie 3 de ce Notebook, lorsque nous ferons des liens d'entités, qu'il existe une entrée Wikipedia pour *Allison McGeer* vers laquelle nous pourrions établir un lien. Nous pouvons considérer la forme plus longue de *Allison McGeer* comme la forme normalisée.

**(TO DO) Q4 (a) - 3 points**  

Vous devez écrire une fonction qui trouvera la forme la plus longue pouvant correspondre à une mention.

Votre fonction aura les mêmes paramètres *entity* et *entities*, mais cette fois la fonction devra attribuer à *entité* le type NER d'une autre entité dans l'itérateur *entities*, soit le NER de la forme la plus longue.

Plus précisément, vous devez parcourir les *entités* pour trouver une forme normalisée de *entité*. Dans ce scénario, l'entité avec la forme la plus longue contenant *entité* en tant que sous-chaîne sera considérée comme la forme normalisée et sera retournée.

Ex : *Lydia Bourouiba* est la forme normalisée de *Bourouiba*. Ainsi, l'entité ayant cette forme doit être retournée. Mais *McMaster University* est déjà la forme la plus longue, donc si nous recherchons une forme normalisée pour cette entité, la fonction devrait renvoyer l'entité elle-même.

In [157]:
# RÉPONSE Q4(a)
# Find the longest surface form within "entities" for which the surface for of "entity" is a substring
def assign_normalized_form(entity, entities):
    tmp = entity
    for token in entities:
        res = token.text.find(entity.text) # -1 = not found
        if(len(token.text) > len(tmp.text) and res != -1):
            tmp = token
    #entity.label_ = tmp.label_
    return tmp  

Testons la fonction ci-haut, en supposant que les candidats se retrouvent uniquement dans les mentions précédentes, car souvent une forme longue est d'abord donnée *Allison McGeer* et les formes subséquentes sont les formes courtes *McGeer*.

In [158]:
# Testing using only the previous references as candidates
test = df["text"][1]
# Parse the text with spaCy
test_sp = sp(test)
for i, token in enumerate(test_sp.ents):
    ent = assign_normalized_form(test_sp.ents[i], test_sp.ents[0:i-1])
    print(str(i) + ": \"" + token.text + "\" is a " + token.label_ + "  " + ent.text + "  " + ent.label_)

0: "weekly" is a DATE  weekly  DATE
1: "Saturday" is a DATE  Saturday  DATE
2: "morning" is a TIME  morning  TIME
3: "two metres" is a QUANTITY  two metres  QUANTITY
4: "COVID-19" is a PERSON  COVID-19  PERSON
5: "the World Health Organization" is a ORG  the World Health Organization  ORG
6: "WHO" is a ORG  WHO  ORG
7: "more than one metre" is a QUANTITY  more than one metre  QUANTITY
8: "the Public Health Agency" is a ORG  the Public Health Agency  ORG
9: "Canada" is a GPE  Canada  GPE
10: "at least two metres" is a QUANTITY  at least two metres  QUANTITY
11: "two" is a CARDINAL  two metres  QUANTITY
12: "2 metres" is a QUANTITY  2 metres  QUANTITY
13: "the 19th century" is a DATE  the 19th century  DATE
14: "1934" is a DATE  1934  DATE
15: "W.F. Wells" is a PERSON  W.F. Wells  PERSON
16: "the Harvard School of Public Health" is a ORG  the Harvard School of Public Health  ORG
17: "two metres" is a QUANTITY  at least two metres  QUANTITY
18: "Wells" is a ORG  W.F. Wells  PERSON
19: "56

**(TO DO) Q4(b) - 2 points**

Faites d'autres tests sans vous limiter à chercher des formes de surfaces plus longues mentionnées avant une entité (voir *test_sp.ents[0:i-1]* dans le code ci-haut), et chercher avant ou après. Ou chercher dans un intervalle (par exemple, max N entités avant ou après). Est-ce que cela fait une différence? Expliquez ce que vous avez testé et fournissez au moins 2 exemples de changements que vous remarquez.

In [167]:
# RÉPONSE Q4(b)
# Do a different test
# This test checks for a normalized form in a range of the word
def assign_normalized_form_range(entity, entities, index):
    tmp = entity
    for token in entities[index-5:index+5]:
        res = token.text.find(entity.text) # -1 = not found
        if(len(token.text) > len(tmp.text) and res != -1):
            tmp = token
    #entity.label_ = tmp.label_
    return tmp  

for i, token in enumerate(test_sp.ents):
    ent = assign_normalized_form_range(test_sp.ents[i], test_sp.ents, i)
    print(str(i) + ": \"" + token.text + "\" is a " + token.label_ + "  " + ent.text + "  " + ent.label_)

0: "weekly" is a DATE  weekly  DATE
1: "Saturday" is a DATE  Saturday  DATE
2: "morning" is a TIME  morning  TIME
3: "two metres" is a QUANTITY  two metres  QUANTITY
4: "COVID-19" is a PERSON  COVID-19  PERSON
5: "the World Health Organization" is a ORG  the World Health Organization  ORG
6: "WHO" is a ORG  WHO  ORG
7: "more than one metre" is a QUANTITY  more than one metre  QUANTITY
8: "the Public Health Agency" is a ORG  the Public Health Agency  ORG
9: "Canada" is a GPE  Canada  GPE
10: "at least two metres" is a QUANTITY  at least two metres  QUANTITY
11: "two" is a CARDINAL  at least two metres  QUANTITY
12: "2 metres" is a QUANTITY  2 metres  QUANTITY
13: "the 19th century" is a DATE  the 19th century  DATE
14: "1934" is a DATE  1934  DATE
15: "W.F. Wells" is a PERSON  W.F. Wells  PERSON
16: "the Harvard School of Public Health" is a ORG  the Harvard School of Public Health  ORG
17: "two metres" is a QUANTITY  two metres  QUANTITY
18: "Wells" is a ORG  W.F. Wells  PERSON
19: "56

**RÉPONSE Q4(b)**
En regardant pour une forme normalisé dans un intervalle de 10 mots (5 en avant et 5 en arrière) on remarque que certains mot comme "Mubareka" et "McMaster" perdent de leur sens. "Samira Mubareka" passe de PERSON à PRODUCT ce qui est faut. Cependant, certains mots comme "U.S." reprennent de leur sens en passant de ORG à GPE. En utilisant un intervalle plutot qu'un le texte au complet, dans certains cas les mots sont corrigé au bon sens, mais dans la majorité des cas la méthode avec intervalle introduit plus d'erreurs que de corrections. 

**(TO DO) Q5 - 5 points**  

Utilisez un autre article de nouvelle dans le corpus, le 7e article, donc index 6.

(a) (2 points) Exécutez les deux approches (NER la plus fréquente, NER de la forme la plus longue). Pour chaque entité trouvée dans le texte, imprimez son type d'entité d'origine (tel que trouvé par spaCy, puis le type d'entité le plus courant (résultat de Q3), puis la forme normalisée avec son type d'entité (résultat de Q4). \
(b) (3 points) Analyser et discuter les résultats. Pensez-vous que ces approches de cohérence de texte aident ou sont-elles trop simples ? Y a-t-il des résultats contradictoires (les deux approches donnent des résultats différents). Si oui, montrez des exemples différents.



In [166]:
# RÉPONSE Q5(a) 
# Select document index 6
doc = df["text"][6]
q5 = sp(doc)

for i, token in enumerate(q5.ents):
    print(str(i) + ": " + token.text)
    print("spaCY: " + token.label_)
    longest = assign_normalized_form(q5.ents[i], q5.ents[0:i-1])
    print("Forme normalisée: " + longest.label_)
    mostFrequent = most_common_type(q5.ents[i], q5.ents)
    print("La plus fréquente: " + mostFrequent)
    print("")

0: Canada
spaCY: GPE
Forme normalisée: ORG
La plus fréquente: GPE

1: C.D. Howe
spaCY: ORG
Forme normalisée: ORG
La plus fréquente: ORG

2: Ontario
spaCY: PERSON
Forme normalisée: PERSON
La plus fréquente: PERSON

3: Monday
spaCY: DATE
Forme normalisée: DATE
La plus fréquente: DATE

4: Alberta
spaCY: GPE
Forme normalisée: GPE
La plus fréquente: GPE

5: first
spaCY: ORDINAL
Forme normalisée: ORDINAL
La plus fréquente: ORDINAL

6: Saturday
spaCY: DATE
Forme normalisée: DATE
La plus fréquente: DATE

7: Air Canada
spaCY: ORG
Forme normalisée: ORG
La plus fréquente: ORG

8: Christmas
spaCY: DATE
Forme normalisée: DATE
La plus fréquente: DATE

9: more than $1.2 million
spaCY: MONEY
Forme normalisée: MONEY
La plus fréquente: MONEY

10: England
spaCY: GPE
Forme normalisée: GPE
La plus fréquente: GPE

11: Peter Cziborra/Reuters
spaCY: PERSON
Forme normalisée: PERSON
La plus fréquente: PERSON

12: months
spaCY: DATE
Forme normalisée: DATE
La plus fréquente: DATE

13: CBC
spaCY: ORG
Forme normali

**RÉPONSE Q5(b) - 3 point**     

Ces approches sont trop simples. Pour l'approche de la forme normalisée, cette approche semble mieux fonctionner pour des noms de personne que celle de spaCY et l'approche de la plus fréquente. Exemple, pour le nom Kenney (76 et 80), spaCY et l'approche de la plus fréquente identifient ce mot comme étant une organisation tandis l'approche de la forme normalisée l'identifie comme étant une personne ce qui est la bonne réponse. Par contre, l'approche de la forme normalisé semble faire plus d'erreurs que l'approche spaCY et l'approche de la plus fréquente pour identifié des entités géopolitiques. Par exemples, le mot Canada et Calgary (0 et 38 respectivement) qui sont identifiés comme ORG par la forme normalisée tandis que la bonne réponse est GPE. On peut donc conclure que ces approches sont trop simples, car elles laissent encore trop d'erreurs.

**PARTIE 3 - Linking d'entité / Enrichissement du texte**  

Pour la troisième partie de ce notebook, nous explorerons comment nous pouvons fournir une "valeur ajoutée" au texte des documents. Dans ce scénario, nous enrichirons le texte en effectuant un linking d'entité. 

Cela signifie que nous tenterons de relier les entités détectées par le NER de spaCy à une page Web active sur laquelle un lecteur peut cliquer pour obtenir plus d'informations concernant l'entité. Wikipedia est une très bonne ressource pour trouver plus d'informations sur une entité, et nous utiliserons cette ressource pour le linking d'entités.

Avant de tenter de faire cette opération automatiquement, voici un exemple de la façon dont un texte sans linking d'entité se compare à un texte enrichi avec linking d'entité fait manuellement:

*Aucun linking d'entité:* \
Pendant la pandémie, des villes américaines telles que Atlanta, Chicago et Denver ont apporté plusieurs ajustements à leurs systèmes de transport en commun.

*Avec linking d'entité:* \
Pendant la pandémie, des villes américaines telles que <a href="http://en.wikipedia.org/wiki/Atlanta"> Atlanta </a>, <a href = "http://en.wikipedia.org/wiki / Chicago "> Chicago </a> et <a href="http://en.wikipedia.org/wiki/Denver"> Denver </a> ont apporté plusieurs ajustements à leurs systèmes de transport en commun.

Transformer un texte automatiquement avec des liens cliquables demande plusieurs traitements au niveau des chaînes de caractères.  Nous nous contenterons dans ce Notebook de trouver les liens sans faire les remplacements directement dans le texte.  Cela nous permettra d'explorer la ressource Wikipedia, et comprendre les difficultés relatives au "entity linking" sans perdre trop de temps dans la manipulation complexe de chaînes de caractères.

Par exemple, avec le document (index 6), nous voudrions pouvoir lier les entités trouvées par spaCy à la page wikipedia la plus probable donnant accès à de l'information supplémentaire sur cette entité.

**Le format ci-bas de liste enrichie est le type de résultat demandé à la question Q6 ci-après.**  Pour simplifier votre code, nous utiliserons ce genre de sortie plutôt que d'avoir les liens directement dans le texte.

0: "Coronavirus Brief" is a ORG found at http://en.wikipedia.org/wiki/Coronavirus_Brief \
1: "CBC" is a ORG found at http://en.wikipedia.org/wiki/CBC \
2: "Canada" is a GPE found at http://en.wikipedia.org/wiki/Canada \
3: "C.D. Howe" is a PERSON found at http://en.wikipedia.org/wiki/C.D._Howe \
4: ... \



**(TO DO) Q6 - 5 points**  \
Écrivez le code nécessaire pour faire la recherche d'une page wikipedia pour les entités trouvées par spaCy (tel qu'illustré ci-haut) dans un document particulier.

*Vous pouvez écrire ce code comme vous le souhaitez, mais il faut inclure les éléments suivants :*

* (a) Une restriction sur le type d'entités que vous liez. Par exemple, Wikipedia ne contient pas de quantités (tel "two meters") il serait alors inadéquat d'inclure un lien vers une quantité.
* (b) L'utilisation de la *forme normalisée* de l'entité pour effectuer la liaison. Par exemple, *Allison McGeer* a une page Wikipédia (https://en.wikipedia.org/wiki/Allison_McGeer) vers laquelle vous pouvez créer un lien, même lorsque vous regardez l'entité avec l'étiquette *McGeer*. Assurez-vous donc d'utiliser la fonction que vous avez développée à la Question 4 (Q4).
* (c) Attention : la page wikipedia utilise des traits de soulignement. Ainsi, par exemple, *McMaster University* doit être transformé en https://en.wikipedia.org/wiki/McMaster_University (avec un trait de soulignement entre *McMaster* et *University*)
* (d) Inclure un élément de post-traitement sur la forme de surface la plus longue trouvée. Par exemple, *the C.D. Howe Institute's* est trouvé par spaCy, mais Wikipédia contiendra *C.D._Howe_Institute*. Vous pouvez supprimer les petites particules comme *the* pour augmenter les chances de lien.
* (e) Pour un document à l'entrée, imprimez une liste des formes de surface, type d'entité et lien vers Wikipedia (tel que montré ci-haut)

Assurez-vous de mettre des commentaires dans votre code pour indiquer clairement ce qui correspond aux parties (a), (b), (c), (d) et (e).

Il y aura probablement de nombreux liens qui mènent vers des pages Wikipedia inexistante.  C'est bon, ne vous inquiétez pas pour ça. Wikipédia ne contient pas tout, et certaines formes normalisées n'y seront pas. On vous demandera de discuter les résultats du linking à la question 7.


In [210]:
# RÉPONSE Q6 -
docq6 = df["text"][6]
q6 = sp(docq6)

for i, token in enumerate(q6.ents):
    # (b) using normalized form
    ent = assign_normalized_form(q5.ents[i], q5.ents[0:i-1])
    # (a) 
    if(ent.label_ == "QUANTITY" or ent.label_ == "CARDINAL"):
        # (e) printing surface form, entity type
        print(str(i) + ": \"" + ent.text + "\" is a " + ent.label_)
    else:
        # (d) remove stop words from string
        stop_words = ["more", "than","the","a","about","above","after","again","against","all","am","an","and","any","are","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can't","cannot","could","couldn't","did"]
        res = ent.text
        for word in ent.text.split():
            if(word in stop_words):
                res = res.replace(word, "")
                # remove fornt and back white spaces
                res = res.strip()
                # remove 's form string
                res = res.replace("'s", "")
        # (c) using ent.text.replace(" ", "_") to transform white spaces to underscrores
        res = res.replace(" ", "_")
        # (e) printing surface form, entity type and Wikipedia link 
        print(str(i) + ": \"" + ent.text +"\" is a " + ent.label_ + " found at " + "https://en.wikipedia.org/wiki/" + res)


0: "Health Canada" is a ORG found at https://en.wikipedia.org/wiki/Health_Canada
1: "C.D. Howe" is a ORG found at https://en.wikipedia.org/wiki/C.D._Howe
2: "Ontario" is a PERSON found at https://en.wikipedia.org/wiki/Ontario
3: "Monday" is a DATE found at https://en.wikipedia.org/wiki/Monday
4: "Alberta" is a GPE found at https://en.wikipedia.org/wiki/Alberta
5: "first" is a ORDINAL found at https://en.wikipedia.org/wiki/first
6: "Saturday" is a DATE found at https://en.wikipedia.org/wiki/Saturday
7: "Air Canada" is a ORG found at https://en.wikipedia.org/wiki/Air_Canada
8: "Christmas" is a DATE found at https://en.wikipedia.org/wiki/Christmas
9: "more than $1.2 million" is a MONEY found at https://en.wikipedia.org/wiki/$1.2_million
10: "England" is a GPE found at https://en.wikipedia.org/wiki/England
11: "Peter Cziborra/Reuters" is a PERSON found at https://en.wikipedia.org/wiki/Peter_Cziborra/Reuters
12: "months" is a DATE found at https://en.wikipedia.org/wiki/months
13: "CBC" is a

**(TO DO) Q7 - 5 points**  

Effectuez une évaluation qualitative de la méthode de linking d'entités que vous avez écrite en Q6. Pour votre évaluation qualitative, vous devez choisir un document (celui que vous voulez dans le corpus de nouvelles sur Covid-19, et assurez-vous de mentionner lequel) et exécuter votre méthode sur ce document. Répondez aux questions suivantes :

* a. Donnez 2 exemples d'entités où la forme plus longue a été trouvée dans Wikipedia. La page trouvée est-elle appropriée ? La forme plus courte serait-elle également trouvée? Serait-elle liée à la même page?
* b. Donnez 2 exemples d'entités où la page wikipedia n'existait pas. Pourquoi? La forme recherchée était-elle incorrecte ou l'entité était-elle peu connue (et donc sans page Wikipedia)?
* c. Essayez de restreindre votre recherche avec différents types d'entités. Pensez-vous que les DATE sont couvertes par Wikipédia ? Qu'en est-il de PERSON ou de GPE ? Discutez de la couverture des différents types d'entités en donnant des exemples.

**RÉPONSE Q7**
Pour cette partie, j'ai utilisé la document 7 soit l'index 6.

a) Pour l'entité "Air Canada", la forme courte était "Canada" (entité 55). La page trouvé pour cette entité est appropriée tandis que si on aurait utilisé la forme plus courte on aurait trouvé une autre page web soit celle du Canada, une entité complètement différente.

Même chose pour l'entité "Jason Kenney" (76). La forme courte de cette entité était "Kenney". La page trouvé pour cette entité est appropriée tandis que si on aurait utilisé la forme plus courte on aurait trouvé aucune page web.

b) Pour l'entité "Grace Horsfall Couldwell" (116) aucune page wikipedia n'existait. Ceci est du au fait que cette personne est très peu connu, donc elle n'a pas de page wikipedia, même si le nom est bien écrit.

Pour l'entité "the entire month" (61) aucune page wikipedia n'existait, car l'entité recherché dans wikipedia était "entire_month". La forme recherché était donc incorrecte. La bonne forme aurait été "month".

c) Pour la plupart des entités DATE, elles sont couvertes par wikipedia sauf si ces dates sont de mauvaise forme ("the entire month", entité (61)) ou référent à des événements historiques ("100 days", entité 135).

Pour les entités de type PERSON ou GPE, la plupart sont couvert sauf si la personne ou l'entité géopolitique n'est pas très connu ou bien cette personne ou cette entité géopolitique est dans un mauvais format. En grande majorité, la couverture est très grande.


***SIGNATURE:***
Mon nom est Mark-Olivier Poulin.
Mon numéro d'étudiant(e) est 300058025.
J'atteste être l'auteur(e) de ce Notebook.